In [1]:
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
import pprint

In [2]:
df = pd.DataFrame(columns = ['news'])
df

,news


## Generating Data of Real News.

In [3]:
l = ["world","business","technology","showbuzz","sports","science","health","trending-news","magazine","lifestyle","newsmo","education-today","crime"]

In [4]:
url = 'https://www.indiatoday.in/'

response = requests.get(url) # getting the url

soup = BeautifulSoup(response.content, 'html.parser')
tabs = soup.find_all("a")

dup_check = []
links = []

for tab in tabs:
  href = tab.get("href")
  if href != None:
    if href.startswith("https://www.indiatoday.in/"):      
      for title in l:
        if (title in href) and ('visualstories' not in href) and (title not in dup_check):
          dup_check.append(title)
          links.append(href)
          print(href)


https://www.indiatoday.in/magazine
https://www.indiatoday.in/world
https://www.indiatoday.in/business
https://www.indiatoday.in/technology
https://www.indiatoday.in/showbuzz
https://www.indiatoday.in/sports
https://www.indiatoday.in/science
https://www.indiatoday.in/health
https://www.indiatoday.in/trending-news
https://www.indiatoday.in/lifestyle
https://www.indiatoday.in/newsmo
https://www.indiatoday.in/education-today
https://www.indiatoday.in/crime


In [5]:

for lin_no in range(len(links)):
  response = requests.get(links[lin_no])
  soup = BeautifulSoup(response.content, 'html.parser')
  articles = soup.find_all('article')

  for art_no in range(len(articles)):
    sub_link = articles[art_no].find('a')
    article_href = sub_link.get("href")
    article_url = 'https://www.indiatoday.in/' + article_href 

    article_resp = requests.get(article_url)
    article_soup = BeautifulSoup(article_resp.content, 'html.parser')

    scripts = article_soup.find_all('script', type='application/ld+json')
    try:
      for i in range(len(scripts)):
        data = json.loads(scripts[i].string)
        if (type(data) == dict):
          if data['@type'].lower() == 'NewsArticle'.lower():
            tempdf = pd.DataFrame(data,index = [0])
            df = pd.concat([df,tempdf],ignore_index =True)
    except:
      None  

In [6]:
df = df.drop_duplicates()
df.reset_index(drop= True,inplace = True)

In [15]:
df.to_excel('conten_2.xlsx')

## Fakenews data.

In [8]:
f_links = []

In [9]:
fakedf = pd.DataFrame(columns = ['news'])
fakedf

,news


In [10]:
url = 'https://thefauxy.com/'

response = requests.get(url) # getting the url

soup = BeautifulSoup(response.content, 'html.parser')
tabs = soup.find_all("a")

dup_check = []
links = []
c= 0
for tab in tabs:
  if tab != None:
    if 'menuitem' in str(tab):
      if c==5:
        break
      else:
        c+=1
        href = tab.get("href")
        print(href)
        f_links.append(href)

for i in range(25):
  href = 'https://thefauxy.com/page/'+str(i)+'/'
  print(href)
  f_links.append(href)

https://thefauxy.com/politics/
https://thefauxy.com/global/
https://thefauxy.com/entertainment/
https://sports.thefauxy.com
https://thefauxy.com/business/
https://thefauxy.com/page/0/
https://thefauxy.com/page/1/
https://thefauxy.com/page/2/
https://thefauxy.com/page/3/
https://thefauxy.com/page/4/
https://thefauxy.com/page/5/
https://thefauxy.com/page/6/
https://thefauxy.com/page/7/
https://thefauxy.com/page/8/
https://thefauxy.com/page/9/
https://thefauxy.com/page/10/
https://thefauxy.com/page/11/
https://thefauxy.com/page/12/
https://thefauxy.com/page/13/
https://thefauxy.com/page/14/
https://thefauxy.com/page/15/
https://thefauxy.com/page/16/
https://thefauxy.com/page/17/
https://thefauxy.com/page/18/
https://thefauxy.com/page/19/
https://thefauxy.com/page/20/
https://thefauxy.com/page/21/
https://thefauxy.com/page/22/
https://thefauxy.com/page/23/
https://thefauxy.com/page/24/


In [11]:
print(len(f_links))

30


In [12]:
for f_linlen in range(len(f_links)):

  d = {0:'politics',1:'global',2:'entertainment', 3: 'sports',4:'business'}
  try:
    articlesection = d[f_linlen]
  except:
    articlesection = 'Nan'

  response = requests.get(f_links[f_linlen])
  soup = BeautifulSoup(response.content, 'html.parser')

  links = []
  for link in soup.find_all("a"):
      href = link.get("href")
      if href is not None:
          links.append(href)

  links = [i for i in links if len(i) > 38]
  links = list(set(links))
  for lin_no in range(len(links)):
    try:
        
      response = requests.get(links[lin_no])
      soup = BeautifulSoup(response.content, 'html.parser')

      title = soup.find('h1', class_='page-title').text.strip()
      date = soup.find('time', class_='ct-meta-element-date')['datetime']

      description = soup.find('div', class_='entry-content').text.strip()
      temp_dict = {'articlesection': articlesection,'Title': title, 'Date': date, 'Description': description}

      filtered_dict = {}
      for key, value in temp_dict.items():
          if len(value) > 0:  
              filtered_dict[key] = value

      tempdf = pd.DataFrame(filtered_dict, index=[0])
      tempdf.fillna('', inplace=True)
      fakedf = pd.concat([fakedf,tempdf],ignore_index =True)
    except:
      None 

In [13]:
fakedf.shape

(411, 5)

In [14]:
fakedf = fakedf.drop_duplicates()
fakedf.reset_index(drop= True,inplace = True)
fakedf

,news,articlesection,Title,Date,Description
0,NaN,politics,Journalist asks The Kerala Story makers to sha...,2023-05-07T15:24:10+05:30,The massive box office success of The has made...
1,NaN,politics,"Imran Khan fools Pakistanis, does a concert pr...",2023-04-09T18:31:25+05:30,"Marshmello played a live set in Karachi, Pakis..."
2,NaN,politics,Jio Cinema buys digital streaming rights of UP...,2023-04-30T10:47:56+05:30,Gangster-turned-politician Atiq Ahmad whose so...
3,NaN,politics,Wrestlers’ protest row: Phogat sisters to retu...,2023-05-04T18:16:30+05:30,Indian wrestlers protesting against Wrestling ...
4,NaN,politics,SC to form Collegium to decide people’s gender,2023-04-30T10:48:01+05:30,During the hearing on the recognition of same-...
...,...,...,...,...,...
391,NaN,Nan,The Kashmir Files Declared Tax Free In Pakistan,2022-03-14T13:29:12+05:30,The Kashmir Files has taken India by storm. Th...
392,NaN,Nan,The Kashmir Files: Yet Another Vivek Agnihotri...,2022-03-13T23:19:37+05:30,Vivek Agnihotri’s The Kashmir Files has opened...
393,NaN,Nan,Starbucks Fires an Employee for Successfully S...,2022-03-15T13:16:06+05:30,A misspelled name on the side of your Starbuck...
394,NaN,Nan,In retaliation to India’s Accidentally Fired M...,2022-03-11T19:27:45+05:30,Pakistan said on Friday that it has retaliated...


In [17]:
df5th = pd.read_excel('/content/collected_news.xlsx')

In [18]:
df = pd.concat([df,df5th],ignore_index =True)

In [19]:
df = df.drop_duplicates()
df.reset_index(drop= True,inplace = True)


,news,@context,@type,mainEntityOfPage,headline,articleSection,url,description,keywords,datePublished,dateModified,articleBody,publisher,hasPart,author,associatedMedia,image,Unnamed: 0,real/fake
0,NaN,http://schema.org/,NewsArticle,https://www.indiatoday.in/magazine/cover-story...,TB: The new dangers,Cover Story,https://www.indiatoday.in/magazine/cover-story...,The rise in the number of cases and deaths due...,"tuberculosis, TB, drug-resistant, India Today ...",2023-05-05T17:34:03+05:30,2023-05-05T19:38:59+05:30,IT was only natural that India took the lead i...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,http://schema.org/,NewsArticle,https://www.indiatoday.in/magazine/editor-s-no...,From the Editor-in-Chief,Editor's Note,https://www.indiatoday.in/magazine/editor-s-no...,The spectre of TB not only continues to haunt ...,"tuberculosis, TB, drug-resistant, India Today ...",2023-05-04T16:42:21+05:30,2023-05-04T16:42:15+05:30,Through the Covid-19 pandemic that inaugurated...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,http://schema.org/,NewsArticle,https://www.indiatoday.in/magazine/up-front/st...,Chhattisgarh Maoist attack | The price of comp...,UP Front,https://www.indiatoday.in/magazine/up-front/st...,District Reserve Guard men have great terrain ...,"DRG, Maoists, Chhattisgarh",2023-05-05T15:09:21+05:30,2023-05-05T15:09:12+05:30,"After a gap of almost four years, a major Maoi...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,http://schema.org/,NewsArticle,https://www.indiatoday.in/magazine/up-front/st...,UP civic polls | Mayawati lobs a googly,UP Front,https://www.indiatoday.in/magazine/up-front/st...,Mayawati’s Muslim outreach has set the alarm b...,"Karnataka polls, HD Kumaraswamy, Siddaramaiah,...",2023-05-05T15:01:40+05:30,2023-05-05T15:01:35+05:30,"On April 24, when Bihar chief minister Nitish ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,http://schema.org/,NewsArticle,https://www.indiatoday.in/magazine/up-front/st...,Saurashtra Tamils | From Salem to Somnath,UP Front,https://www.indiatoday.in/magazine/up-front/st...,The sangamam is officially to connect Saurasht...,"sangamam, Saurashtra Tamils, homeland, Salem, ...",2023-05-05T13:46:21+05:30,2023-05-05T13:46:16+05:30,A millennium-old conn&shy;ec&shy;tion was revi...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
444,NaN,http://schema.org/,NewsArticle,https://www.indiatoday.in/crime/story/bihar-ma...,"Bihar: 7, including minor girl, held for purch...",Crime,https://www.indiatoday.in/crime/story/bihar-ma...,Sources revealed the gang misused the procedur...,"cybercrime, crime syndicate in Bihar, Bihar on...",2023-04-29T22:35:49+05:30,2023-04-29T22:35:45+05:30,Bihar&rsquo;s Economic Offences Unit (EOU) arr...,NaN,NaN,NaN,NaN,NaN,230.0,NaN
445,NaN,http://schema.org/,NewsArticle,https://www.indiatoday.in/crime/story/tamil-na...,"40-year-old hacked to death by 3 men, body lef...",Crime,https://www.indiatoday.in/crime/story/tamil-na...,A man was hacked to death by three men in Tami...,"tamil nadu hacked to death, man hacked to deat...",2023-04-29T20:33:27+05:30,2023-04-29T20:33:22+05:30,A 40-year-old man was hacked to death in broad...,NaN,NaN,NaN,NaN,NaN,231.0,NaN
446,NaN,http://schema.org/,NewsArticle,https://www.indiatoday.in/crime/story/tripura-...,"Head in 1 bag, rest in another: Tripura man ki...",Crime,https://www.indiatoday.in/crime/story/tripura-...,A Tripura man allegedly killed his minor wife ...,"tripura man kills minor wife, minor wife kille...",2023-04-29T19:44:07+05:30,2023-04-29T19:44:02+05:30,A 15-year-old girl was allegedly murdered by h...,NaN,NaN,NaN,NaN,NaN,232.0,NaN
447,NaN,http://schema.org/,NewsArticle,https://www.indiatoday.in/crime/story/man-arre...,"Murder accused kills live-in partner in Delhi,...",Crime,https://www.indiatoday.in/crime/story/man-arre...,A 26-year-old man was arrested by Delhi police...,"arrest, murder, delhi news, delhi, crime, rela...",2023-04-29T15:13:50+05:30,2023-04-29T15:13:42+05:30,A 26-year-old man was arre

In [20]:
df.to_excel('real_news_2days_data.xlsx')
fakedf.to_excel('fake_news_data.xlsx')
